# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [3]:
import pandas as pd
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [4]:
df.sample(20) #clean dataset

,المخالفة,الغرامة
42,قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.,الغرامة المالية 500 - 900 ريال
83,استخدام لوحات غير صادرة من الإدارة المختصة.,الغرامة المالية 1000 - 2000 ريال
38,عدم إعطاء الأفضلية للمركبة القادمة من اليمين ع...,الغرامة المالية 500 - 900 ريال
67,عدم إخراج المركبة المعدة للتصدير خلال المدة ال...,الغرامة المالية 1000 - 2000 ريال
39,عدم إعطاء الأفضلية للمركبات التي على الطريق ال...,الغرامة المالية 500 - 900 ريال
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
22,عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة...,الغرامة المالية 300 - 500 ريال
26,استعمال المكابح (الفرامل) بشكل مفاجئ لغير ضرورة.,الغرامة المالية 300 - 500 ريال
43,زيادة أبعاد الحمولة المنقولة لمركبات النقل الخ...,الغرامة المالية 500 - 900 ريال
103,المجازفة بعبور الأودية والشعاب أثناء جريانها.,الغرامة المالية 5000 - 10000 ريال


In [5]:
df.tail()

,المخالفة,الغرامة
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال
103,المجازفة بعبور الأودية والشعاب أثناء جريانها.,الغرامة المالية 5000 - 10000 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [6]:
df.columns

Index(['المخالفة', 'الغرامة'], dtype='object')

In [7]:
#merge columns as one text
text = df['المخالفة'] + ' ' + df['الغرامة']

In [8]:
text

,0
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرته...
2,عدم وجود تأمين ساري للمركبة. الغرامة المالية 1...
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم. ...
4,عدم تقيد المشاة بالإشارات الخاصة بهم. الغرامة ...
...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...
101,استخدام لوحات غير عائدة للمركبة. الغرامة المال...
102,طمس رقم هيكل المركبة أو محاولة طمسه. الغرامة ا...


In [9]:
text = ','.join(text)

In [10]:
text[:100]

'قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال,ترك المركبة مفتو'

In [11]:
text

'قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. الغرامة المالية 100 - 150 ريال,عدم وجود تأمين ساري للمركبة. الغرامة المالية 100 - 150 ريال,عبور المشاة للطرق من غير الأماكن المخصصة لهم. الغرامة المالية 100 - 150 ريال,عدم تقيد المشاة بالإشارات الخاصة بهم. الغرامة المالية 100 - 150 ريال,وقوف المركبة في أماكن غير مخصصة للوقوف. الغرامة المالية 100 - 150 ريال,عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. الغرامة المالية 100 - 150 ريال,عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار. الغرامة المالية 150 - 300 ريال,الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً. الغرامة المالية 150 - 300 ريال,قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر. الغرامة المالية 150 - 300 ريال,مخالفة قواعد استعمال أنوار التلاقي. الغرامة المالية 150 - 300 ريال,عدم حمل رخصة القيادة أو رخ

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [12]:
!pip install langchain

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
#aaaaaaaaaaaaaaaaaaa,aaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa,

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[","],
    chunk_size=100,
    chunk_overlap=10,
    length_function=len
)

In [16]:
# from langchain.docstore.document import Document

In [17]:
# toy = 'قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. الغرامة المالية 100 - 150 ريال,عدم وجود تأمين ساري للمركبة. الغرامة المالية 100 - 150 ريال,عبور المشاة للطرق من غير الأماكن المخصصة لهم. الغرامة المالية 100 - 150 ريال,عدم تقيد المشاة بالإشارات الخاصة بهم. الغرامة المالية 100 - 150 ريال,وقوف المركبة في أماكن غير مخصصة للوقوف. الغرامة المالية 100 - 150 ريال,عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. الغرامة المالية 100 - 150 ريال,عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار. الغرامة المالية 150 - 300 ريال'

In [18]:
# documents = [Document(page_content=toy)]
# documents

In [19]:

# text_splitter.split_documents(documents)

In [20]:
#split our text

text = text_splitter.split_text(text)

In [21]:
text

['قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال',
 ',ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. الغرامة المالية 100 - 150 ريال',
 ',عدم وجود تأمين ساري للمركبة. الغرامة المالية 100 - 150 ريال',
 ',عبور المشاة للطرق من غير الأماكن المخصصة لهم. الغرامة المالية 100 - 150 ريال',
 ',عدم تقيد المشاة بالإشارات الخاصة بهم. الغرامة المالية 100 - 150 ريال',
 ',وقوف المركبة في أماكن غير مخصصة للوقوف. الغرامة المالية 100 - 150 ريال',
 ',عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. الغرامة المالية 100 - 150 ريال',
 ',عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار. الغرامة المالية 150 - 300 ريال',
 ',الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً. الغرامة المالية 150 - 300 ريال',
 ',قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر. الغرامة المالية 150 - 300 ريال',
 ',مخالفة قواعد استعمال أنوار التلاقي. الغرام

In [97]:
len(text)

104

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [22]:
!pip install langchain_huggingface

In [23]:
pip install -qU "langchain-chroma>=0.1.2" #https://python.langchain.com/docs/integrations/vectorstores/chroma/

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 9

In [24]:

from transformers import AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt


EMBEDDING_MODEL_NAME = "akhooli/personachat-arabic"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-25-28e67f2c6d43>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to 

In [101]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="traffic_violation",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)
docs = [Document(page_content=t) for t in text]
vector_store = vector_store.from_documents(docs,embedding_model)

In [29]:
!pip install -U langchain-community

## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [166]:
from langchain_core.prompts import PromptTemplate


PROMPT_TEMPLATE="""
انت ذكاء اصطناعي على علم بالمخالفات المرورية
 بحيث يستفسر منك المستخدم عن غرامة المخالفة
، اجب من باللغه العربية و من {context}  فقط
في حال كان السؤال من خارج {context} قل للمستخدم لا أعلم
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [32]:
!pip install langchain_groq

In [155]:
import getpass
import os

groq_api_key = ""


In [156]:
%pip install -qU langchain-groq

In [167]:
from langchain_groq import ChatGroq

chatgroq = ChatGroq(
   # model_name="akhooli/personachat-arabic",
    temperature=0,
    api_key=groq_api_key,
)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [158]:
 from langchain.chains import LLMChain

In [168]:
 chain = LLMChain(llm=chatgroq, prompt=prompt_template)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [169]:
query_embedding = embedding_model.embed_query("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟")

relevant_docs = vector_store.similarity_search_by_vector(query_embedding, k=3)

In [161]:
relevant_docs[0]

Document(metadata={}, page_content=',عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية. الغرامة المالية 500 - 900 ريال')

In [173]:
def query_rag(query: str):
    similarity_search_results = vector_store.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = chain.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [174]:
query_rag("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟")

{'context': ',عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية. الغرامة المالية 500 - 900 ريال\n\n,عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية. الغرامة المالية 500 - 900 ريال\n\n,عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية. الغرامة المالية 500 - 900 ريال\n\n,عدم الوقوف تماماً عند إشارة قف. الغرامة المالية 500 - 900 ريال',
 'question': 'ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟',
 'text': 'انتبه لا تتم الوقوف تاماً عند إشارة (أمامك أفضلية) أثناء مرور مركبات على الطريق المعطاة لها الأفضلية, غرامتك ستكون بين 500 و 900 ريال. ولا تنس أن عدم الوقوف تماماً عند إشارة قف يجب أن يكون أيضاً مسؤوليتك. ولكن إذا كنت تسأل عن غير ذلك، فلا أستطيع أن أعطي لك جواباً.'}

In [ ]:
#!rm -rf /content/chroma_langchain_db

In [133]:
#!pip install gradio
#import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.0.1
    Uninstalling websockets-13.0.1:
      Successfully uninstalled websockets-13.0.1
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [176]:
demo = gr.Interface(
    fn=query_rag,
    inputs=gr.Textbox(lines=0, placeholder="نظام المرور التقليدي"),
    outputs="text",

)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c7fc833a240bc7184.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
